Semantic Segmentation

In [ ]:
! git clone https://github.com/sunghoonYoon/Kitti_TA

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
Kitti_options = NONE #SS: Semantic Segmentation , SM: Stereo Matching, OF: Optical flow /// Only these three options are available

assert Kitti_options in ['SS','SM','OF']

from PIL import Image
from torchvision import transforms
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch
import numpy as np

from collections import OrderedDict
import time

########
from Kitti_TA.kitti_seg import decode_segmap, denorm, YOUR_IMPLEMENTATION ##import segmentation network (deeplabWplus)
from Kitti_TA.kitti_stereo import YOUR_IMPLEMENTATION #import stereo matching network


left_image_path = '/content/drive/MyDrive/TA_stereo/KITTI_2015/testing/left.png'
right_image_path = '/content/drive/MyDrive/TA_stereo/KITTI_2015/testing/right.png'




In [ ]:
if Kitti_options == 'SS':
    net_SS = YOUR_IMPLEMENTATION.cuda() # declare Network and the number of class=19
    state_dict= torch.load('/content/drive/MyDrive/TA_segmentation/cityscapes_best.pth')
    net_SS = torch.nn.DataParallel(net_SS)
     
    net_SS.YOUR_IMPLEMENTATION(state_dict['state_dict'],strict=False) #load the checkpoint(state_dict) 

    net_SS.eval()

    image= np.asarray(Image.open(left_image_path)).convert('RGB')
    
    MEAN = [0.45734706, 0.43338275, 0.40058118]
    STD = [0.23965294, 0.23532275, 0.2398498]

    toTensor = transforms.ToTensor()
    normTensor = transforms.Normalize(MEAN,STD)
    imageT = normTensor(toTensor(image))

    with torch.no_grad():
        
        imageT = F.interpolate(imageT.unsqueeze(0), size=(512,1024),mode='bilinear',align_corners=True)
        YOUR_IMPLEMENTATION # get segmentation results using network

        YOUR_IMPLEMENTATION # Take softmax to the network output in channel dimension
        YOUR_IMPLEMENTATION # Get max indices from the softmax-ed result (use argmax function)
    
    segmap = YOUR_IMPLEMENTATION #use the function that we provide (convert prediction into RBG color map)

    ###Visualization###
    '''
    YOUR IMPLEMENTATION
    #Visualize the segmentation result and image together with transparency (alpha=0.7) mode. use plt function

    ''' 

Stereo matching

In [ ]:
if Kitti_options == 'SM':

In [ ]:
    max_disp = 192
    net_SM = YOUR_IMPLEMENTATION # declare Network and the max disparity is 192
    net_SM = net_SM.cuda()
    net_SM.eval()

In [ ]:
    checkpoint = torch.load('/content/drive/MyDrive/TA_stereo/pretrained_model_KITTI2015.tar')
    new_state_dict = OrderedDict()
    for k, v in checkpoint['state_dict'].items():
        name = k[7:]
        new_state_dict[name] = v    
    net_SM.load_state_dict(new_state_dict)    
    normal_mean_var = {'mean': [0.485, 0.456, 0.406],
                        'std': [0.229, 0.224, 0.225]}
    infer_transform = transforms.Compose([transforms.ToTensor(),
                                            transforms.Normalize(**normal_mean_var)])    


    imgL_o = Image.open(left_image_path).convert('RGB')
    imgR_o = Image.open(right_image_path).convert('RGB')

    imgL = infer_transform(imgL_o)
    imgR = infer_transform(imgR_o) 

    # pad to width and hight to 16 times
    if imgL.shape[1] % 16 != 0:
        times = imgL.shape[1]//16       
        top_pad = (times+1)*16 -imgL.shape[1]
    else:
        top_pad = 0

    if imgL.shape[2] % 16 != 0:
        times = imgL.shape[2]//16                       
        right_pad = (times+1)*16-imgL.shape[2]
    else:
        right_pad = 0    

    imgL = F.pad(imgL,(0,right_pad, top_pad,0)).unsqueeze(0)
    imgR = F.pad(imgR,(0,right_pad, top_pad,0)).unsqueeze(0)

    start_time = time.time()
    
    net_SM.eval()

    imgL = imgL.cuda()
    imgR = imgR.cuda()  
           

    with torch.no_grad():
        disp = YOUR_IMPLEMENTATION # get output from the network

        disp = torch.squeeze(disp)
        pred_disp = disp.data.cpu().numpy()


    if top_pad !=0 or right_pad != 0:
        img = pred_disp[top_pad:,:-right_pad]
    else:
        img = pred_disp

    ####Visualize####
    '''
    YOUR IMPLEMENTATION
    #Visualize the left image and left disparity using plt function.

    '''

    